In [1]:
import numpy as np
import torch

In [2]:
import sys
sys.path.append(sys.path[0] + "/..")
import models.bar_embedding as bemb

In [10]:
import importlib
importlib.reload(bemb)

<module 'models.bar_embedding' from '/home/ignacy/dev/py/mir/exploration/../models/bar_embedding.py'>

In [24]:
parts = np.load("../data/measures/esac_parts.npy", allow_pickle=True)
MIN_PITCH = 48
MAX_PITCH = 94
DUR_SIZE = 25

In [51]:
def lower_pitch_to_2(measure: torch.Tensor) -> torch.Tensor:
    measure = measure.clone()
    pitches = measure[:, 0]
    true_pitch_idxs = pitches > 1
    pitches[true_pitch_idxs] -= MIN_PITCH - 2
    measure[:, 0] = pitches
    return measure

def pad_bar(bar: torch.Tensor, to_pad:int=16, pad_value=0.0) -> torch.Tensor:
    bar_len = bar.shape[0]
    pad_len = to_pad - bar_len
    return torch.nn.functional.pad(
        bar, pad=(0, 0, 0, pad_len), mode="constant", value=pad_value
    )

def encode_measure(measure: tuple[tuple[int, int]]) -> torch.Tensor:
    m = torch.tensor(measure)
    return pad_bar(lower_pitch_to_2(m))

In [54]:
def octavise_bar(bar: torch.Tensor) -> torch.Tensor:
    """
    bar.shape <=> (num_of_notes, 2) pitch and duration
    new_bar.shape <=> (num_of_notes, 3) octave, note in octave, duration
    """

    new_bar = torch.zeros((bar.shape[0], 3), dtype=bar.dtype)
    true_pitch_idxs = bar[:, 0] > 1

    new_bar[~true_pitch_idxs, 0:2] = bar[~true_pitch_idxs, 0].unsqueeze(dim=1)

    SPECIAL_PITCH_COUNT = 2

    new_bar[true_pitch_idxs, 0] = (
        (bar[true_pitch_idxs, 0] - SPECIAL_PITCH_COUNT) // 12
    ) + SPECIAL_PITCH_COUNT
    new_bar[true_pitch_idxs, 1] = (
        bar[true_pitch_idxs, 0] - SPECIAL_PITCH_COUNT
    ) % 12 + SPECIAL_PITCH_COUNT
    new_bar[:, 2] = bar[:, 1]

    return new_bar

In [55]:
tensor_parts = []

for part in parts:
    tensor_parts.append(
        torch.stack(
            [encode_measure]
        )
    )
encode_measure(parts[4][2])

tensor([[21, 10],
        [20,  7],
        [20,  7],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0],
        [ 0,  0]])

In [40]:
bar_encoder = bemb.Encoder(
    pitch_vocab_count=50,
    duration_vocab_count=25,
    pitch_embedding_dim=16,
    duration_embedding_dim=16,
    bar_len=16,
    bar_embedding_len=4,
    out_dim=64
)

bar_decoder = bemb.Decoder(
    in_dimension=64,
    octave_vocab=6,
    bar_len=16,
    duration_vocab=25,
)

PATH = "../models/serialized/bar_embedding/"
bar_encoder.load_state_dict(torch.load(f"{PATH}encoder-v01.pt")) 
bar_decoder.load_state_dict(torch.load(f"{PATH}decoder-v01.pt"))

<All keys matched successfully>

In [50]:
bar = parts[10][0]
print(encode_measure(bar).T)
ebar = bar_encoder.forward(encode_measure(bar).unsqueeze(dim=0))
o, n, d = bar_decoder.forward(ebar)
print(torch.stack([o.argmax(dim=2), n.argmax(dim=2), d.argmax(dim=2)]).swapaxes(0, 1))

tensor([[14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
tensor([[[3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])


In [12]:
ds = bemb.Dataset(list(set(bar for part in encoded_parts for bar in part)))

AttributeError: 'tuple' object has no attribute 'shape'